In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import unicodedata
import re
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import json

page_limit = 1000
#json stuffs
with open('tennis.json') as f:
    Data = json.load(f)
    names = [datum['name'] for datum in Data]

#Specify the initial page to crawl
base_url = 'http://comp20008-jh.eng.unimelb.edu.au:9889/main/'
seed_items = ''
patten = r'(\(?.?\d(-|/)\d\)?.?){2,}'
#patten = r'(( ((\(\d-\d\))|(\d-\d))( |.|,|\w)))'
#patten = '\w (((\d-\d)+(\(\d-\d\))+) )+\w|.|,s' 
#the first page
seed_url = base_url + seed_items
page = requests.get(seed_url)
soup = BeautifulSoup(page.text, 'html.parser')
output = open("task2.csv","w")
output.write("player,url,score,title\n")
#if find the first page, set up the visiting condition
visited = {}; 
visited[seed_url] = True
pages_visited = 1

#parse the first page
body= soup.find('body')
content = str(body.text.strip())
content_length = len(content)
for name in names:
    break_flag = 0
    name_length = len(name)
    for x in range(content_length-name_length-1):
        if name.lower() == content[x:x+name_length].lower():
            print(name)
            break_flag = 1
            break;
    if(break_flag):
        break           
#display(HTML(str(content)))
#check valid function
#Remove index.html
links = soup.findAll('a')
seed_link = soup.findAll('a', href=re.compile("^index.html"))
#to_visit_relative = list(set(links) - set(seed_link))
to_visit_relative = [l for l in links if l not in seed_link]
def isvalid(score):
    score = score.replace("/","-").replace("(","").replace(")","")
    each_rounds = score.split(" ")
    each_rounds.remove("")
    for each_round in each_rounds:
        each_round = each_round.split("-")
        scorea = int(each_round[0])
        scoreb = int(each_round[1])
        if(scorea < 6 and scoreb < 6):
            return False
        if((scorea == 0 or scoreb == 0)and(scorea > 7 or scoreb>7 )):
            return False
    return True
        

# Resolve to absolute urls
booktitle = []
to_visit = []
for link in to_visit_relative:
    to_visit.append(urljoin(seed_url, link['href']))

    
#Find all outbound links on succsesor pages and explore each one 
while (to_visit):
    # Impose a limit to avoid breaking the site 
    if pages_visited == page_limit :
        break
        
    # consume the list of urls
    link = to_visit.pop(0)


    # need to concat with base_url, an example item <a href="catalogue/sharp-objects_997/index.html">
    page = requests.get(link)
    
    # scarping code goes here
    soup = BeautifulSoup(page.text, 'html.parser')
    body= soup.find('body')
    content = str(body.text.strip())
    content_length = len(content)
    #find the name and the score of that player
    break_flag = 0
    for x in range(content_length):
        for name in names:
            name_length = len(name)
            if((name_length + x) > content_length):
                break;
            if name.lower() == content[x:x+name_length].lower():
                score = re.search(patten,content[x:])
                if score:
                    score = score.group(0)
                    purified_score = ""
                    for element in score.split(" "):
                        if element != '':
                            element = element.replace(".","").replace(",","")
                            purified_score = purified_score + element + " "
                    if isvalid(purified_score):
                        print(purified_score)
                        output.write(name + ","+link+","+purified_score+",")
                        heads = soup.find_all('h1')
                        for head in heads:
                            output.write("\""+str(head.text.strip())+"\"")
                        output.write("\n")                        
                    purified_score = ""
                    break_flag = 1
                    break;
        if(break_flag):
            break    

    # mark the item as visited, i.e., add to visited list, remove from to_visit
    visited[link] = True
    new_links = soup.findAll('a')
    for new_link in new_links :
        new_item = new_link['href']
        new_url = urljoin(link, new_item)
        if new_url not in visited and new_url not in to_visit:
            to_visit.append(new_url)
        
    pages_visited = pages_visited + 1
output.close()

4-6 7-6 (8-6) 6-4 
5-7 7-5 7-6 
7-6 (7-2) 6-4 
6-4 7-5 
7-6 (7-3) 6-3 
7-5 6-4 
7-5 6-3 
6-2 6-2 
6-3 6-1 
4-6 6-3 7-6 
7-6 (7-3) 6-3 
7-5 6-3 
6-3 6-1 
6-7 6-4 6-4 
7-5 6-1 
6-4 7-5 
7-5 6-4 
6-3 6-7 (2-7) 7-6 (7-5) 
1-6 6-3 6-4 6-4 
6-1 6-1 
6-3 6-4 
7-6 (7-4) 6-3 
7-5 6-0 
3-6 6-4 7-6 (7/5) 
6-4 6-4 
7-5 6-3 
3-6 7-6 (7-3) 7-6 (7-4) 6-1 
6-4 6-1 
6-3 6-2 1-6 3-6 10-8 
7-5 6-3 
7-6 (7-4) 6-3 
6-1 6-2 
7-5 7-6 (7-3) 6-1 
7-5 6-3 3-6 6-3 
6-4 7-6 (7-4) 
6-2 7-6 (7-1) 7-6 (7-5) 
6-0 6-3 6-2 
6-1 7-5 
6-2 7-6 (7-1) 7-6 (7-5) 
6-2 7-6 (7-1) 7-6 (7-5) 
